( klines['Close'].max() / klines['Close'].min() ) range for how much it has fluctuated before buying, if it has gone down alot or already gone up alot

In [95]:
from sys import displayhook
from binance.client import Client
from binance.enums import *
import pandas as pd
import time
import datetime
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt

def getminutedata(symbol, start, end):
    frame = pd.DataFrame(client.get_historical_klines(symbol, Client.KLINE_INTERVAL_15MINUTE, start, end))
    frame = frame.iloc[:,:6]                                                    
    frame.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']
    frame = frame.set_index('Time')
    frame.index = pd.to_datetime(frame.index, unit= 'ms')
    frame = frame.astype(float)
    return frame.reset_index()

api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)

In [88]:

api_key = 'ihvXWbHvvDrYuroIJOS9ANW6X7tv5OQSYibLLloKyIGui0w93Q5yUJOBZclnNL5D'
api_secret = 'uzmKBJ3lmVS4TsTl2AOHVJOYTFfLM5qrg00vAqUyTYbSjiy7BDPlTpaq2Oy7qdo2'
client = Client(api_key, api_secret)
db_data = 'mysql+mysqldb://' + 'admin' + ':' + '0323A8E3DB' + '@' + 'coin-database-1-instance-1.c9hqcydvu19f.us-east-1.rds.amazonaws.com' + ':3306/' + 'crypto_db' + '?charset=utf8mb4'
engine = create_engine(db_data)

check_date = pd.read_sql_query("SELECT MAX(date_request) FROM crypto_trend", engine) 
date_recent_checked = str(check_date.iloc[0, 0])

trend_table_status = pd.read_sql_query("show table status like 'crypto_trend'", engine)
trend_table_rowcount_old = trend_table_status['Rows'].values[0]

cryptoString = 'SELECT * FROM crypto'
crypto_table = pd.read_sql_query(cryptoString, engine)
full_id_list = crypto_table.id.astype(int).tolist()



In [122]:
start_string = "2022-03-11 14:00:00"
end_string = "2022-03-17 21:00:00"
start_date = datetime.datetime.strptime(start_string, '%Y-%m-%d %H:%M:%S')
end_date = datetime.datetime.strptime(end_string, '%Y-%m-%d %H:%M:%S') 
for i in range(int((end_date - start_date).total_seconds()/3600)):
    date_lower = str(start_date + i*datetime.timedelta(hours=1))
    date_upper = str(start_date + (i+1)*datetime.timedelta(hours=1))
    print(date_lower)
    print(date_upper)

2022-03-11 14:00:00
2022-03-11 15:00:00
2022-03-11 15:00:00
2022-03-11 16:00:00
2022-03-11 16:00:00
2022-03-11 17:00:00
2022-03-11 17:00:00
2022-03-11 18:00:00
2022-03-11 18:00:00
2022-03-11 19:00:00
2022-03-11 19:00:00
2022-03-11 20:00:00
2022-03-11 20:00:00
2022-03-11 21:00:00
2022-03-11 21:00:00
2022-03-11 22:00:00
2022-03-11 22:00:00
2022-03-11 23:00:00
2022-03-11 23:00:00
2022-03-12 00:00:00
2022-03-12 00:00:00
2022-03-12 01:00:00
2022-03-12 01:00:00
2022-03-12 02:00:00
2022-03-12 02:00:00
2022-03-12 03:00:00
2022-03-12 03:00:00
2022-03-12 04:00:00
2022-03-12 04:00:00
2022-03-12 05:00:00
2022-03-12 05:00:00
2022-03-12 06:00:00
2022-03-12 06:00:00
2022-03-12 07:00:00
2022-03-12 07:00:00
2022-03-12 08:00:00
2022-03-12 08:00:00
2022-03-12 09:00:00
2022-03-12 09:00:00
2022-03-12 10:00:00
2022-03-12 10:00:00
2022-03-12 11:00:00
2022-03-12 11:00:00
2022-03-12 12:00:00
2022-03-12 12:00:00
2022-03-12 13:00:00
2022-03-12 13:00:00
2022-03-12 14:00:00
2022-03-12 14:00:00
2022-03-12 15:00:00


In [121]:
(end_date - start_date).total_seconds()/3600

151.0

In [147]:
#setup variables
maxholds = 3
trades = [[],[],[],[]]
tradedata = []
buyPrice  = [0] * (len(full_id_list))
sellPrice = [0] * (len(full_id_list))
tradeBuy  = [0] * (len(full_id_list))
tradeSell = [0] * (len(full_id_list))
buyTime   = [0] * (len(full_id_list))
sellTime  = [0] * (len(full_id_list))
holding = [False] * (len(full_id_list))
buy_ids = []
buy_threshold = 80
sell_threshold = 50


start_string = "2022-03-11 14:00:00"
end_string = "2022-03-17 21:00:00"
start_date = datetime.datetime.strptime(start_string, '%Y-%m-%d %H:%M:%S')
end_date = datetime.datetime.strptime(end_string, '%Y-%m-%d %H:%M:%S') 
for i in range(int((end_date - start_date).total_seconds()/3600)):
    date_lower = str(start_date + i*datetime.timedelta(hours=1))
    date_upper = str(start_date + (i+1)*datetime.timedelta(hours=1))
    date_price_lower = str(start_date + i*datetime.timedelta(hours=1) - datetime.timedelta(hours=5))
    ############################################ TESTING ONLY ##################################################################
    new_trends_string = f"SELECT * FROM crypto_trend WHERE date_request > '{date_lower}' AND date_request < '{date_upper}' ORDER BY date_request, crypto_id, date"
    new_trends = pd.read_sql_query(new_trends_string, engine)
    ####################################################################################################################################

    #update to ensure that if the count increased inbetween the request for count and request for data, does not re-check
    trend_table_rowcount_old = pd.read_sql_query("show table status like 'crypto_trend'", engine)['Rows'].values[0]

    if new_trends.empty:
        print(f"Dataframe empty for request: {new_trends_string}")
        check_data = False
        waiting = True
        
    date_request = new_trends[['crypto_id', 'date_request']].groupby(['crypto_id']).mean()
    date_recent_checked = new_trends.date_request.max()

    try:
        trends = new_trends.pivot(index="date", columns="crypto_id", values="hype") #unbeliiiiieveevevevable !! https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html
    except:
        pd.set_option("display.max_rows", None, "display.max_columns", None)
        print(new_trends)
        

    #check buying and selling conditions
    check_100 = ((trends.iloc[-1] == 100) | (trends.iloc[-2] == 100)) ##check trends with current time
    check_non0 = trends.iloc[0:-2].mean() > 7 
    check_peak = (trends.iloc[0:-10] < buy_threshold).all()
    trend_indicate_buy = np.logical_and.reduce((check_100, check_non0, check_peak), dtype=bool)

    curr_id_list = trends.columns.values
    check_upward_ids = [curr_id_list[i] for i in [i for i, x in enumerate(trend_indicate_buy) if x]]


    buy_ids = []
    if check_upward_ids:
        #print(f"Indicate buy ids: {check_upward_ids} between {date_last_checked} : {date_recent_checked}")
        for key, crypto_id in enumerate(check_upward_ids):
            value = getminutedata(crypto_table.loc[crypto_table['id'] == crypto_id]['ticker'].values[0], 
                                                str(date_request.loc[crypto_id]['date_request'] - datetime.timedelta(hours=2)) ,
                                                str(date_request.loc[crypto_id]['date_request']) )
            close = value[['Time','Close']]                             
            close.index = pd.to_datetime(close['Time'])
            close = close.drop(columns=['Time'])
            close = close.rename(columns={"Close": f"{crypto_id}"})
            
            if (key != 0):
                closes = closes.join(close)
            else:
                closes = close

        increase = closes.diff(periods=6)
        check_increase = (increase.iloc[-1] >= 0)

        indicate_buy = check_increase[check_increase].index.values.astype(int)
        #print(f"Indicate buy ids: {indicate_buy} between {date_last_checked} : {date_recent_checked}")

        curr_NOT_holding_ids = [full_id_list[i] for i in [i for i, x in enumerate(np.logical_not(holding)) if x]]
        buy_ids = list(set(indicate_buy).intersection(curr_NOT_holding_ids))

    check_decrease = np.logical_or((trends.iloc[-1] < sell_threshold), (trends.iloc[-2] < sell_threshold))
    indicate_sell = check_decrease[check_decrease].index.values.astype(int)
    #print(f"Indicate sell ids: {indicate_sell} between {date_last_checked} : {date_recent_checked}")

    curr_holding_ids = [full_id_list[i] for i in [i for i, x in enumerate(holding) if x]]
    sell_ids = list(set(indicate_sell).intersection(curr_holding_ids))

    # Buy control
    if ((sum(holding) < maxholds) and buy_ids):
        for id in buy_ids:
            if (sum(holding) < maxholds):
                key = full_id_list.index(id)
                tradetime = closes.index[-1]
                buyPrice[key] = closes.loc[tradetime,str(id)]
                buyTime[key] = tradetime
                holding[key] = True
                #print(f"Purchased {crypto_table.loc[crypto_table['id'] == id]['ticker'].values[0]} id {id}  at {buyPrice[key]} - {tradetime}")
            else:
                continue
            
    # Sell control
    if (sell_ids):
        for id in sell_ids:
            value = getminutedata(crypto_table.loc[crypto_table['id'] == id]['ticker'].values[0], 
                                                str(date_request.loc[crypto_id]['date_request'] - datetime.timedelta(hours=2)) ,
                                                str(date_request.loc[crypto_id]['date_request']) )
            close = value[['Time','Close']]                             
            close.index = pd.to_datetime(close['Time'])
            close = close.drop(columns=['Time'])
            # Sell execute function binance using ticker
            key = full_id_list.index(id)
            tradetime = close.index[-1]
            sellPrice[key] = close.loc[tradetime, 'Close']
            sellTime[key] = tradetime
            print(f"Trade completed: {crypto_table.loc[crypto_table['id'] == id]['ticker'].values[0]} id = {id} | open = {buyPrice[key]} at {buyTime[key]} | close = {sellPrice[key]} at {sellTime[key]} | profit = {(sellPrice[key]/buyPrice[key])}%")

            # upload to database
            quantity = 0
            trade = (id, buyTime[key], buyPrice[key], sellTime[key], sellPrice[key], str(sellTime[key] - buyTime[key]),  sellPrice[key]/buyPrice[key], quantity)
            index_names = ['crypto_id', 'date_buy', 'price_buy', 'date_sell', 'price_sell', 'trade_time', 'trade_return', 'quantity']
            result = pd.DataFrame([trade],columns=index_names)
            #print(result)
            tradedata.append([id, buyTime[key], sellTime[key], buyPrice[key], sellPrice[key], (sellPrice[key]/buyPrice[key]) ])
            holding[key] = False


Trade completed: IRISUSDT id = 144 | open = 0.0577 at 2022-03-11 14:00:00 | close = 0.05647 at 2022-03-11 15:00:00 | profit = 0.978682842287695%
Trade completed: WOOUSDT id = 286 | open = 0.489 at 2022-03-11 15:00:00 | close = 0.4742 at 2022-03-11 16:00:00 | profit = 0.9697341513292433%
Trade completed: USTUSDT id = 272 | open = 1.004 at 2022-03-11 16:00:00 | close = 1.0042 at 2022-03-11 17:00:00 | profit = 1.000199203187251%
Trade completed: ETCUSDT id = 104 | open = 26.2 at 2022-03-11 18:00:00 | close = 26.23 at 2022-03-11 19:00:00 | profit = 1.001145038167939%
Trade completed: FARMUSDT id = 106 | open = 87.3 at 2022-03-11 18:00:00 | close = 87.0 at 2022-03-11 19:00:00 | profit = 0.9965635738831615%
Trade completed: FETUSDT id = 107 | open = 0.3405 at 2022-03-11 20:00:00 | close = 0.342 at 2022-03-11 21:00:00 | profit = 1.0044052863436124%
Trade completed: MBOXUSDT id = 171 | open = 1.81 at 2022-03-11 20:00:00 | close = 1.806 at 2022-03-11 22:00:00 | profit = 0.9977900552486187%
Trad

In [146]:
date_request = new_trends[['crypto_id', 'date_request']].groupby(['crypto_id']).mean()
date_request
#str(date_request.loc[300]['date_request'])

,date_request
crypto_id,
1,2022-03-17 20:02:03
2,2022-03-17 20:02:14
3,2022-03-17 20:02:25
4,2022-03-17 20:02:36
5,2022-03-17 20:02:47
...,...
300,2022-03-17 20:10:42
301,2022-03-17 20:10:52
302,2022-03-17 20:11:03


In [149]:
# Calculate profit
tradedata = np.array(tradedata)
scaled = ((tradedata[:,5] - 1) * 1/maxholds) + 1
scaled.prod()

0.8694298637673121

In [148]:
tradedata = np.array(tradedata)
tradedata[:,5].prod()

0.6355674133307827

first test = 0.8694298637673121 (not equal to 0.65)
Changing time to time of trend. (just realised this was scaled and not actual, assume if it is scaled it is closer to 1 and less frequent big wins / losses are les impactful ?)
Second test = 